# L2b: What does the largest eigenpair represent?
Now that we know how to compute the largest eigenpair of a matrix, let's explore what these eigenvalues and eigenvectors represent. 

> __Learning Objectives:__
>
> By the end of this lab, you will be able to:
>
> * __Construct transition matrices from graph adjacency matrices:__ Convert edge lists into adjacency matrices and normalize them to create probability transition matrices for random walks.
> * __Apply power iteration to find the largest eigenpair:__ Use the power iteration algorithm to compute the dominant eigenvalue and corresponding eigenvector of a transition matrix.
> * __Interpret stationary distributions as PageRank scores:__ Explain how the eigenvector corresponding to the largest eigenvalue represents webpage importance in the PageRank algorithm.


Let's get started!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up our code environment:

In [1]:
include(joinpath(@__DIR__, "Include.jl")); # include the Include.jl file

In addition to standard Julia libraries, we'll also use [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl). Check out [the documentation](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/) for more information on the functions, types, and data used in this material.

### Data
Next, let's load up the dataset that we will explore. This dataset was generated with the help of generative AI and a simple randomized graph generator for teaching and demonstration purposes. 

It does not contain real hyperlinks, real traffic patterns, or data collected from any website. Any resemblance to real domains is coincidental (the domain-like labels are fabricated).

We've provided [the `MySyntheticPageRankDataset()` function](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/data/#VLDataScienceMachineLearningPackage.MySyntheticPageRankDataset) to load the synthetic PageRank dataset. This function takes no arguments and returns a tuple containing the edges and nodes of the synthetic web graph.

> __What's in the dataset?__
> 
> The dataset contains two data structures: `edges::Dict{Int, Tuple{String, String}}` maps edge indices to pairs of node identifiers (from_node, to_node), representing directed hyperlinks between webpages. The `nodes::Dict{String, NamedTuple}` maps node identifiers to named tuples containing metadata such as page labels, community assignments, and page types.

Let's load the dataset:

In [2]:
(edges, nodes) = MySyntheticPageRankDataset(); # load the synthetic PageRank dataset

Let's take a look at the `edges::Dict{Int, Tuple{String, String}}` dictionary?

In [3]:
edges

Dict{Int64, Tuple{String, String}} with 2564 entries:
  1144 => ("p0096", "p0104")
  2108 => ("p0188", "p0170")
  1175 => ("p0096", "p0219")
  1953 => ("p0172", "p0142")
  719  => ("p0065", "p0087")
  1546 => ("p0129", "p0126")
  1703 => ("p0143", "p0157")
  1956 => ("p0172", "p0147")
  2261 => ("p0197", "p0201")
  2288 => ("p0200", "p0197")
  1028 => ("p0089", "p0118")
  699  => ("p0063", "p0073")
  831  => ("p0075", "p0045")
  1299 => ("p0108", "p0105")
  1438 => ("p0119", "p0092")
  1074 => ("p0093", "p0110")
  2350 => ("p0205", "p0209")
  2493 => ("p0221", "p0224")
  319  => ("p0023", "p0000")
  ⋮    => ⋮

How about the `nodes::Dict{String, NamedTuple}` dictionary?

In [4]:
nodes

Dict{String, NamedTuple} with 242 entries:
  "p0020" => (nodeid = "p0020", label = "ledger-ne20.com", community = "news", …
  "p0065" => (nodeid = "p0065", label = "review-sc20.net", community = "science…
  "p0229" => (nodeid = "p0229", label = "deal-09-clickbait.com", community = "s…
  "p0217" => (nodeid = "p0217", label = "pop-en42.io", community = "entertainme…
  "p0126" => (nodeid = "p0126", label = "sportsbook-sp36.io", community = "spor…
  "p0070" => (nodeid = "p0070", label = "journal-sc25.com", community = "scienc…
  "p0194" => (nodeid = "p0194", label = "reel-en19.io", community = "entertainm…
  "p0123" => (nodeid = "p0123", label = "locker-sp33.net", community = "sports"…
  "p0156" => (nodeid = "p0156", label = "fund-fi26.io", community = "finance", …
  "p0068" => (nodeid = "p0068", label = "lab-sc23.org", community = "science", …
  "p0200" => (nodeid = "p0200", label = "reel-en25.org", community = "entertain…
  "p0022" => (nodeid = "p0022", label = "bulletin-ne22.net", commu

Finally, let's compute a few things we'll need below, in particular the list of nodes and the number of nodes.

In [5]:
number_of_nodes, list_of_nodes = let

    # initialize -
    nodeset = Set{String}();
    number_of_edges = keys(edges) |> length;

    # loop over edges to build the set of nodes. 
    # Trick: Take advantage of the fact that sets do not allow duplicates (nice!)
    for i ∈ 1:number_of_edges
        (from_node, to_node) = edges[i];
        push!(nodeset, from_node);
        push!(nodeset, to_node);
    end

    # Of course, we want a sorted array of nodes (not a set), so let's convert to an array and sort it.
    list_of_nodes = nodeset |> collect |> sort;
    number_of_nodes = length(list_of_nodes); # how many nodes are there?

    (number_of_nodes, list_of_nodes); # return 
end

(242, ["p0000", "p0001", "p0002", "p0003", "p0004", "p0005", "p0006", "p0007", "p0008", "p0009"  …  "p0232", "p0233", "p0234", "p0235", "p0236", "p0237", "p0238", "p0239", "p0240", "p0241"])

In [6]:
number_of_nodes

242

___

## Task 1: Compute the transition matrix
In this task, we will compute the transition matrix for the synthetic web graph dataset. To do this, first lets's convert our edge list into an adjacency matrix. Then, we will compute the transition matrix from the adjacency matrix.

> __What is an adjacency matrix?__
> 
> An adjacency matrix is a square matrix used to represent a finite graph. The elements of the matrix indicate whether pairs of vertices are adjacent or not in the graph. Suppose our adjancey matrix is given by $\mathbf{A}$. Then, the element $A_{ij}$ is non-zero if there is an edge from node $i$ to node $j$. In particular, for an unweighted graph, $A_{ij} = 1$ if there is an edge from node $i$ to node $j$, and $A_{ij} = 0$ otherwise.

Let's build the adjacency matrix as a sparse matrix to save memory. A sparse matrix is a matrix in which most of the elements are zero. Storing only the non-zero elements can save significant memory and computational resources, especially for large matrices. 

Julia provides support for sparse matrices through its standard library module, [SparseArrays](https://docs.julialang.org/en/v1/stdlib/SparseArrays/#stdlib-sparse-arrays). 

In [7]:
A = let

    # initialize -
    A = spzeros(Int, number_of_nodes, number_of_nodes); # sparse adjacency matrix

    # ok, loop over edges to populate the adjacency matrix
    for (i, (from_node, to_node)) ∈ edges
        from_index = findfirst(isequal(from_node), list_of_nodes);
        to_index   = findfirst(isequal(to_node), list_of_nodes);
        A[from_index, to_index] = 1; # unweighted graph
    end
    
    A # return
end;

In [8]:
A

242×242 SparseMatrixCSC{Int64, Int64} with 2564 stored entries:
⎡⣵⡿⣿⣿⢸⣿⣽⡆⠠⠠⠄⠀⡀⡄⠀⠈⢠⣅⠄⠠⠠⠠⠄⠆⠀⠀⠀⠀⠀⠄⡄⢄⢄⠀⠤⠠⠆⠀⠀⠀⎤
⎢⣿⣿⡯⡞⣿⡿⣿⡇⠀⠐⠀⢀⢒⠢⣅⠠⠀⠌⠌⠀⠀⢀⠐⡃⠐⠀⠀⠰⠀⡀⠀⠀⠄⠁⠀⠀⠀⠀⠀⠀⎥
⎢⣷⣷⣶⣷⢼⣯⣶⣇⠄⢀⠄⠀⢠⠀⡠⠂⠠⠄⠈⠠⠀⠄⠄⠆⡐⠀⠀⠀⠀⠈⠠⠐⢀⠐⠀⠀⠀⠀⠀⠀⎥
⎢⠟⠿⠿⠾⢤⠿⠿⢃⣂⡀⣀⠄⣀⣀⣀⠐⠀⠀⠐⠀⠀⠄⠀⠀⠀⠀⠀⠀⠐⠀⠀⠀⠀⠀⠀⠄⠀⠀⡀⠀⎥
⎢⠀⠑⡦⠀⠀⣁⠬⢱⣴⡯⣏⣵⣼⡧⡼⠀⠠⠀⠀⢀⠀⠀⠀⠀⠈⠠⠀⠀⠰⢀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⢘⠀⠡⠀⡁⠀⢸⣽⡯⣕⣫⣿⣯⣺⠀⠀⡀⠀⠀⠀⠀⡀⠠⠀⠀⠀⠀⠐⠀⢀⠀⠀⠀⠀⠂⠀⠀⠀⠀⎥
⎢⠀⢐⠈⠂⠀⠈⠀⢸⣢⣽⣟⣻⢼⡿⣿⠀⠀⠅⠀⠀⠀⠐⠅⠀⠠⠀⠀⠠⠀⠂⠀⠀⠈⠀⠀⠄⠀⠀⠀⠀⎥
⎢⡢⣿⠦⠁⢀⣗⢕⠞⠚⢞⢟⠻⢟⢻⢺⣧⢤⣶⣤⣴⣤⣶⠆⢤⡐⢤⣂⠄⠷⠄⢈⡤⣔⢀⣄⠄⡆⠄⠐⡀⎥
⎢⠀⡐⠤⠔⠠⠡⠰⠀⠀⡀⠤⠁⠀⠄⡂⣿⣻⣿⣧⣿⡽⡓⠠⠃⠀⠄⠀⠀⠂⠀⠀⠀⠀⠀⠠⠀⠀⠀⠀⠀⎥
⎢⠐⠄⠖⠐⠄⡃⡀⠀⠀⠆⡀⠀⢠⡄⢂⣿⣿⣿⣯⡾⣿⡇⠀⠀⠀⠄⠀⠀⠀⠀⠀⠀⠂⠁⠀⠀⡀⠀⠀⠀⎥
⎢⠀⠀⠢⠄⠀⠥⠰⠌⢀⠀⠀⠁⠀⡠⠀⠿⢼⡿⠏⠿⠿⢇⣀⣀⣁⣀⡀⢀⡀⠀⠠⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⡀⡆⠄⠈⠀⡅⠀⡀⠀⡊⢈⠈⠉⠁⣀⡀⠅⡄⠉⠈⡀⢘⣯⡿⣿⣏⡃⣿⣟⠀⠀⠀⠠⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢈⡐⠀⠀⠂⠀⠁⠀⠔⡀⠀⠀⠂⠣⠂⠀⠀⠂⠀⠀⡀⢸⣿⣷⣿⣮⠏⢏⡢⠀⠁⠀⡀⠀⠂⠀⠀⠀⠀⠀⎥
⎢⠈⠀⠀⡀⠀⠀⠀⠂⠀⠀⠀⠀⠀⠌⠀⡀⠀⠄⠀⢈⠀⢰⣟⣿⠿⣿⡤⠸⣁⠀⠀⢀⠀⠀⠈⠀⠀⠀⠀⠀⎥
⎢⠁⢁⠉⠈⠁⠃⠈⢙⠐⠍⠀⠩⢈⠁⠉⠉⠊⠀⠀⠀⠡⠉⠋⠋⠉⠉⠋⠉⢩⡔⢴⢮⢤⠶⣪⣅⡇⠀⠁⠀⎥
⎢⡅⢨⠀⠌⠤⠀⢄⠤⠄⠠⠤⠨⠀⠬⠀⠥⠀⠤⠤⠄⠀⠴⠤⠤⠁⠄⢄⢠⣼⣯⣾⣭⣾⣧⣿⣭⡇⠁⠀⠀⎥
⎢⠀⡆⠀⠀⢀⠠⠀⢠⠀⠀⠀⠀⠀⠁⠄⠁⠀⠀⠄⠀⠢⠄⠀⠂⠀⠈⠀⠀⢬⡽⣽⢥⣼⠿⣿⡿⡇⠀⠀⠀⎥
⎢⠁⠆⠠⠀⠂⠀⠄⠀⠀⠈⠀⠀⢀⠂⡈⠄⠀⠀⠀⠀⠂⢀⠀⠄⠐⠂⠁⠀⢘⣯⢻⣍⢭⣽⣿⡧⡇⠀⠀⠀⎥
⎢⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠈⠈⠀⠈⠁⠀⠁⠼⣾⠗⡓⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢵⠻⢑⠴⎦

> __What is a transition matrix?__
> 
> A transition matrix $\mathbf{P}$ is a row-stochastic matrix where each entry $P_{ij}$ represents the probability of transitioning from state $i$ to state $j$ in a discrete-time Markov chain. For a web graph, $P_{ij}$ is the probability that a random surfer currently at webpage $i$ follows a link to webpage $j$. The transition matrix is constructed from the adjacency matrix $\mathbf{A}$ by normalizing each row: if webpage $i$ has $k$ outgoing links, then $P_{ij} = A_{ij}/k$ for each connected page $j$. This ensures each row sums to one, $\sum_{j=1}^{n}P_{ij} = 1$ for all $i$, making it a valid probability distribution. Webpages with no outgoing links (dangling nodes) are handled by distributing probability uniformly across all pages: $P_{ij} = 1/n$ for all $j$.

Let's compute the transition matrix:

In [9]:
P = let

    # initialize -
    P = spzeros(Float64, number_of_nodes, number_of_nodes); # sparse transition matrix

    # loop over rows of A to compute the transition matrix
    for i ∈ 1:number_of_nodes
        row_sum = sum(A[i, :]); # sum of the i-th row
        if row_sum != 0
            P[i, :] .= A[i, :] ./ row_sum; # normalize the row (fancy! what is .= doing here?)
        else
            P[i, :] .= 1.0 / number_of_nodes; # handle dangling nodes (no outgoing edges)
        end
    end

    P # return
end

242×242 SparseMatrixCSC{Float64, Int64} with 4500 stored entries:
⎡⣵⡿⣿⣿⢸⣿⣽⡆⠠⠠⠄⠀⡀⡄⠀⠈⢠⣅⠄⠠⠠⠠⠄⠆⠀⠀⠀⠀⠀⠄⡄⢄⢄⠀⠤⠠⠆⠀⠀⠀⎤
⎢⣿⣿⡯⡞⣿⡿⣿⡇⠀⠐⠀⢀⢒⠢⣅⠠⠀⠌⠌⠀⠀⢀⠐⡃⠐⠀⠀⠰⠀⡀⠀⠀⠄⠁⠀⠀⠀⠀⠀⠀⎥
⎢⣷⣷⣶⣷⢼⣯⣶⣇⠄⢀⠄⠀⢠⠀⡠⠂⠠⠄⠈⠠⠀⠄⠄⠆⡐⠀⠀⠀⠀⠈⠠⠐⢀⠐⠀⠀⠀⠀⠀⠀⎥
⎢⠟⠿⠿⠾⢤⠿⠿⢃⣂⡀⣀⠄⣀⣀⣀⠐⠀⠀⠐⠀⠀⠄⠀⠀⠀⠀⠀⠀⠐⠀⠀⠀⠀⠀⠀⠄⠀⠀⡀⠀⎥
⎢⠀⠑⡦⠀⠀⣁⠬⢱⣴⡯⣏⣵⣼⡧⡼⠀⠠⠀⠀⢀⠀⠀⠀⠀⠈⠠⠀⠀⠰⢀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠉⢙⠉⠩⠉⡉⠉⢹⣽⡯⣝⣫⣿⣯⣻⠉⠉⡉⠉⠉⠉⠉⡉⠩⠉⠉⠉⠉⠙⠉⢉⠉⠉⠉⠉⠋⠉⠉⠉⠉⎥
⎢⠀⢐⠈⠂⠀⠈⠀⢸⣢⣽⣟⣻⢼⡿⣿⠀⠀⠅⠀⠀⠀⠐⠅⠀⠠⠀⠀⠠⠀⠂⠀⠀⠈⠀⠀⠄⠀⠀⠀⠀⎥
⎢⡢⣿⠦⠁⢀⣗⢕⠞⠚⢞⢟⠻⢟⢻⢺⣧⢤⣶⣤⣴⣤⣶⠆⢤⡐⢤⣂⠄⠷⠄⢈⡤⣔⢀⣄⠄⡆⠄⠐⡀⎥
⎢⠉⡙⠭⠝⠩⠩⠹⠉⠉⡉⠭⠉⠉⠍⡋⣿⣻⣿⣯⣿⡽⡛⠩⠋⠉⠍⠉⠉⠋⠉⠉⠉⠉⠉⠩⠉⠉⠉⠉⠉⎥
⎢⠴⠤⠶⠴⠤⡧⡤⠤⠤⠦⡤⠤⢤⡤⢦⣿⣿⣿⣯⡾⣿⡧⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠦⠥⠤⠤⡤⠤⠤⠤⎥
⎢⠀⠀⠢⠄⠀⠥⠰⠌⢀⠀⠀⠁⠀⡠⠀⠿⢼⡿⠏⠿⠿⢇⣀⣀⣁⣀⡀⢀⡀⠀⠠⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⡀⡆⠄⠈⠀⡅⠀⡀⠀⡊⢈⠈⠉⠁⣀⡀⠅⡄⠉⠈⡀⢘⣯⡿⣿⣏⡃⣿⣟⠀⠀⠀⠠⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣈⣐⣀⣀⣂⣀⣁⣀⣔⣀⣀⣀⣂⣣⣂⣀⣀⣂⣀⣀⣀⣸⣿⣷⣿⣮⣏⣏⣢⣀⣁⣀⣀⣀⣂⣀⣀⣀⣀⣀⎥
⎢⠾⠶⠶⡶⠶⠶⠶⠶⠶⠶⠶⠶⠶⠾⠶⡶⠶⠶⠶⢾⠶⢶⣿⣿⠿⣿⡶⠾⣷⠶⠶⢶⠶⠶⠾⠶⠶⠶⠶⠶⎥
⎢⠁⢁⠉⠈⠁⠃⠈⢙⠐⠍⠀⠩⢈⠁⠉⠉⠊⠀⠀⠀⠡⠉⠋⠋⠉⠉⠋⠉⢩⡔⢴⢮⢤⠶⣪⣅⡇⠀⠁⠀⎥
⎢⡅⢨⠀⠌⠤⠀⢄⠤⠄⠠⠤⠨⠀⠬⠀⠥⠀⠤⠤⠄⠀⠴⠤⠤⠁⠄⢄⢠⣼⣯⣾⣭⣾⣧⣿⣭⡇⠁⠀⠀⎥
⎢⠉⡏⠉⠉⢉⠩⠉⢩⠉⠉⠉⠉⠉⠉⠍⠉⠉⠉⠍⠉⠫⠍⠉⠋⠉⠉⠉⠉⢭⡽⣽⢭⣽⠿⣿⡿⡏⠉⠉⠉⎥
⎢⠓⠖⠲⠒⠒⠒⠖⠒⠒⠚⠒⠒⢒⠒⡚⠖⠒⠒⠒⠒⠒⢒⠒⠖⠒⠒⠓⠒⢚⣿⢻⣟⢿⣿⣿⡷⡗⠒⠒⠒⎥
⎢⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠈⠈⠀⠈⠁⠀⠁⠼⣾⠗⡓⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢵⠻⢑⠴⎦

__Check__: If this is correct, then each row of the transition matrix should sum to one. You can verify this by summing the rows of the transition matrix and checking if they equal one.

In [10]:
let

    # initialize -
    number_of_nodes = size(P, 1);

    for i ∈ 1:number_of_nodes
        row_sum = sum(P[i, :]);
        @assert isapprox(row_sum, 1.0; atol=1e-8) "Row $i does not sum to 1, sum = $row_sum";
    end
end

In [11]:
P[1,:]

242-element SparseVector{Float64, Int64} with 12 stored entries:
  [7 ]  =  0.0833333
  [11]  =  0.0833333
  [13]  =  0.0833333
  [16]  =  0.0833333
  [19]  =  0.0833333
  [20]  =  0.0833333
  [27]  =  0.0833333
  [30]  =  0.0833333
  [31]  =  0.0833333
  [37]  =  0.0833333
  [40]  =  0.0833333
  [96]  =  0.0833333

## Task 2: Compute the largest eigenpair of the transition matrix
In this task, we will compute the largest eigenpair of the transition matrix $\mathbf{P}$ using the power iteration method.

> __Connection to Markov Processes:__

> The algorithm iterates until the change in the eigenvector estimate (measured by the $\ell_{1}$-norm) falls below a tolerance of $\epsilon = 10^{-8}$. Let's run the power iteration:

> The transition matrix $\mathbf{P}$ defines a discrete-time Markov chain on the web graph. A central question in Markov theory is whether a stationary distribution exists: a probability vector $\boldsymbol{\pi}$ such that $\boldsymbol{\pi}^{\top}\mathbf{P} = \boldsymbol{\pi}^{\top}$, meaning the distribution remains unchanged after one step of the random walk. Transposing both sides gives the eigenvalue equation $\mathbf{P}^{\top}\boldsymbol{\pi} = \boldsymbol{\pi}$, showing that $\boldsymbol{\pi}$ is an eigenvector of $\mathbf{P}^{\top}$ with eigenvalue $\lambda = 1$. The Perron-Frobenius theorem for stochastic matrices guarantees that $\lambda = 1$ is the largest eigenvalue (in magnitude), and for an irreducible, aperiodic chain, the corresponding eigenvector has all positive entries and is unique up to scalar multiplication. This eigenvector, when normalized to sum to one, is the unique stationary distribution representing the long-run fraction of time a random surfer spends at each webpage.

> For a matrix $\mathbf{P}$, we distinguish between right eigenvectors satisfying $\mathbf{P}\mathbf{v} = \lambda\mathbf{v}$ and left eigenvectors satisfying $\mathbf{v}^{\top}\mathbf{P} = \lambda\mathbf{v}^{\top}$. The left eigenvector equation can be rewritten as $\mathbf{P}^{\top}\mathbf{v} = \lambda\mathbf{v}$ by transposing both sides. The stationary distribution is a left eigenvector because it satisfies $\boldsymbol{\pi}^{\top}\mathbf{P} = \boldsymbol{\pi}^{\top}$, describing a distribution that is invariant under the transition dynamics. To compute it using power iteration (which finds right eigenvectors), we transpose $\mathbf{P}$ and solve $\mathbf{P}^{\top}\mathbf{v} = \mathbf{v}$.

> __Left versus Right Eigenvectors:__>

In [27]:
λ̂,v̂ = let

    # initialize -
    max_iterations = 1000;
    tolerance      = 1e-8;
    v = rand(number_of_nodes); # random initial eigenvector
    v .= v ./ norm(v, 1);      # normalize
    A = transpose(P) |> Matrix;  # we want the left eigenvector, so we work with the transpose

    # call the power iteration method
    result = poweriteration(A, v; maxiter = max_iterations, ϵ = tolerance);

    (result.value, result.vector) # return
end

Converged in 31 iterations


(1.0000068472367856, [0.13134465564342657, 0.012992025847351435, 0.10260533053108949, 0.011474520931163572, 0.11173862780887117, 0.0625575224662041, 0.26319225632996196, 0.07035109197700304, 0.10734016447518378, 0.03470895403789316  …  0.009950263999003393, 0.006460825333222089, 0.002040729726507196, 0.0020828320290642117, 0.003242097434038535, 0.0017067363452438707, 0.0021996023434866525, 0.0027607235707679824, 0.0029599354447284675, 0.002434414531399812])

What is the largest eigenvalue and corresponding eigenvector of the transition matrix $\mathbf{P}$? Theory tells us that $\hat{λ} = 1$ should be the largest eigenvalue of a transition matrix. Let's see if our computation agrees with this.

In [28]:
@assert isapprox(λ̂, 1.0; atol=1e-4) # adjust atol to find the max permissible error

> __Why normalize the eigenvector?__
> 
> The power iteration returns an eigenvector $\hat{\mathbf{v}}$ satisfying $\mathbf{P}^{\top}\hat{\mathbf{v}} = \hat{\mathbf{v}}$, but eigenvectors are only defined up to scalar multiplication: if $\mathbf{v}$ is an eigenvector, so is $c\mathbf{v}$ for any nonzero scalar $c$. The power iteration algorithm normalizes at each step using the $\ell_{1}$-norm to prevent numerical overflow, but the final vector is not necessarily a probability distribution. To interpret $\hat{\mathbf{v}}$ as a stationary distribution, we require $\sum_{i=1}^{n}\pi_{i} = 1$ so that $\pi_{i}$ represents the fraction of time spent at node $i$. The normalization $\hat{\pi} = \hat{\mathbf{v}}/(\mathbf{1}^{\top}\hat{\mathbf{v}})$ where $\mathbf{1}$ is a vector of ones ensures this property while preserving the relative magnitudes that encode webpage importance.

Let's compute the stationary distribution:

$$$$
\hat{\pi} = \frac{\hat{\mathbf{v}}}{\mathbf{1}^{\top} \hat{\mathbf{v}}}

In [32]:
π̂ = let
    
    # initialize -
    ones_vector = ones(number_of_nodes);
    T = dot(ones_vector, v̂); # normalization factor
    π̂  = v̂ ./ T # return
end

242-element Vector{Float64}:
 0.011599808148978161
 0.0011474011375458993
 0.009061671701772784
 0.0010133814790628146
 0.00986828614429315
 0.005524817552176328
 0.023244034291067365
 0.006213112867110352
 0.009479832342567095
 0.003065348992838228
 ⋮
 0.0005705929486228387
 0.0001802286754298165
 0.0001839469739011107
 0.0002863284238776985
 0.00015073178325172214
 0.00019425963746675585
 0.00024381550673983007
 0.0002614090624701328
 0.00021499726335256992

__Check__: The entries of the stationary distribution $\hat{\pi}$ should sum to one. You can verify this by summing the entries of $\hat{\pi}$.

In [33]:
@assert isapprox(sum(π̂), 1.0; atol=1e-8) # check that the entries sum to one

Ok, but what does this stationary distribution represent in the context of our web graph? 

> __PageRank and Stationary Distribution__
> In the context of PageRank, the stationary distribution $\hat{\pi}$ represents the long-term behavior of a random surfer navigating the web graph. Each entry $\hat{\pi}_i$ in the stationary distribution corresponds to the probability of being at node (webpage) $i$ after a large number of steps in a random walk on the graph.

Let's find the most important webpage:

In [42]:
nodeid = let
    i = argmax(π̂);
    nodeid = list_of_nodes[i];
    println("The most important webpage is: $(list_of_nodes[i]) with PageRank score $(π̂[i])");
    nodeid;
end;

i = 142
The most important webpage is: p0141 with PageRank score 0.02469991255749491


That `nodeid::String` variable holds the identifier of the most important webpage according to the PageRank analysis. What does this correspond to in the original dataset? You can look it up in the `nodes` dictionary.

In [40]:
let
    println(nodes[nodeid])
end

(nodeid = "p0141", label = "promo_cash_now.org", community = "finance", type = "spam_target")


> __Interpreting PageRank Scores:__
> 
> In the PageRank context, "most important" refers to the webpage with the highest score $\pi_{i}$ in the stationary distribution. This score has a precise probabilistic interpretation: it is the fraction of time a random surfer spends at webpage $i$ in the long run, assuming they navigate by uniformly randomly following outgoing links. High PageRank scores indicate structural importance within the network topology. A page achieves high PageRank through one of two mechanisms: receiving many incoming links (high in-degree), or receiving links from other high-PageRank pages (recursive importance). This captures the intuition that important pages are those that many other important pages link to, creating a self-reinforcing notion of authority. Critically, PageRank measures link structure, not content quality or relevance. A page about an obscure topic could have high PageRank if it is well-connected, while a high-quality page might have low PageRank if it lacks incoming links. The algorithm also treats all outgoing links from a page equally, so a link from a page with one outgoing link carries more weight than a link from a page with many outgoing links. This explains why pages that are linked from focused, specialized sources can achieve higher PageRank than pages linked from general directories with hundreds of outbound links.

Let's look at the details of the top $n$ most important webpages [using the `PrettyTables.jl` package](https://github.com/ronisbr/PrettyTables.jl)

In [ ]:
let

    # initialize
    number_of_sites_to_display = 10;
    i = sortperm(π̂; rev=true)[1:number_of_sites_to_display] # indices of the top 10 most important webpages
    df = DataFrame(); # hold the data for the table

    for j ∈ 1:number_of_sites_to_display
        node_index = i[j];
        node_id = list_of_nodes[node_index];
        page_rank_score = π̂[node_index];
        label = nodes[node_id].label;
        community = nodes[node_id].community;
        type = nodes[node_id].type;
        push!(df, (Rank = j, NodeID = node_id, PageName = label, Community = community, Type = type, PageRankScore = page_rank_score));
    end
    
    # make the table -
    pretty_table(
        df;
        backend = :text,
        fit_table_in_display_horizontally = false,
        table_format = TextTableFormat(borders = text_table_borders__compact)
    );
end

 ------- -------- -------------------- ------------------- ------------------- ---------------
   Rank   NodeID             PageName           Community                Type   PageRankScore 
  Int64   String    SubString{String}   SubString{String}   SubString{String}         Float64 
 ------- -------- -------------------- ------------------- ------------------- ---------------
      1    p0141   promo_cash_now.org             finance         spam_target       0.0246999
      2    p0006      gazette-ne06.io                news              portal        0.023244
      3    p0030    bulletin-ne30.org                news           authority       0.0171696
      4    p0130        fund-fi00.net             finance           authority       0.0144213
      5    p0015       daily-ne15.net                news                 hub       0.0136233
      6    p0175         pop-en00.net       entertainment           authority        0.013244
      7    p0078    insights-sc33.org             scienc

___

## Summary
The largest eigenpair of a transition matrix reveals the long-term behavior of random walks on graphs, with the stationary distribution representing the steady-state probabilities of visiting each node.

> __Key Takeaways:__
> 
> * **Transition matrices encode random walk dynamics:** Normalizing an adjacency matrix produces a stochastic matrix where each row represents transition probabilities from one node to its neighbors.
> * **The dominant eigenvector represents steady-state importance:** The eigenvector corresponding to eigenvalue $\lambda = 1$ gives the stationary distribution, which in the PageRank context measures webpage importance based on link structure.
> * **Power iteration efficiently finds the largest eigenpair:** The power iteration algorithm converges to the dominant eigenvector by repeatedly applying the transition matrix, with convergence measured using the $\ell_{1}$-norm.


This mathematical framework provides the foundation for ranking algorithms that assess importance in networked systems.___